In [1]:
from llama_sim import LlamaForCausalLM
#from transformers import LlamaForCausalLM
import torch
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
model_name = "meta-llama/Meta-Llama-3-70B-Instruct"
hf_llm = LlamaForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.bfloat16)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)


/home/zhuominc/anaconda3/envs/dgl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/zhuominc/anaconda3/envs/dgl/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/zhuominc/anaconda3/envs/dgl/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/zhuominc/anaconda3/envs/dgl/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.r

In [29]:
import numpy as np
import random
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True
# 设置随机数种子
setup_seed(20)
hf_llm = hf_llm.eval()
hf_llm.config.K = 13
hf_llm.config.L = 190
hf_llm.config.cache_mode = "anns_es"
hf_llm.config.window = 1
hf_llm.set_sparse_attn(sparse=0.01, random_sparse=0.2, window_size=16, kernel_size=5)

In [30]:
input_text = """
System: You are an assistant. 
User: Hello! I want to test your mathematic ability. Your task is to finish the mathematical task. The problem for you is:
Condition 1 : @<<<v0 = 1>>>@;@<<<assign v1 = 1>>>@;
Condition 2 : @<<<assign v2 = 4>>>@;@<<<assign v3 = 2>>>@;
Condition 3 : @<<<assign v4 = 4>>>@;@<<<assign v5 = 1>>>@;
**Question:**
Can you tell me how many variables are equal to 1 in those assignments in '<<<>>>'? The values of those variables can be used before definition as they are constant. 
Please solve the problem in a step-by-step way.
Assistant: 
"""
input_tokens = tokenizer.encode(input_text, return_tensors="pt").cuda()
print(input_tokens, len(input_tokens[0]))

tensor([[128000,    198,   2374,     25,   1472,    527,    459,  18328,     13,
            720,   1502,     25,  22691,      0,    358,   1390,    311,   1296,
            701,  21651,   5845,     13,   4718,   3465,    374,    311,   6381,
            279,  37072,   3465,     13,    578,   3575,    369,    499,    374,
            512,  10770,    220,     16,    551,    571,  12889,     85,     15,
            284,    220,     16,  20761,     31,     26,     31,  12889,   7128,
            348,     16,    284,    220,     16,  20761,     31,    280,  10770,
            220,     17,    551,    571,  12889,   7128,    348,     17,    284,
            220,     20,  20761,     31,     26,     31,  12889,   7128,    348,
             18,    284,    220,     17,  20761,     31,    280,  10770,    220,
             18,    551,    571,  12889,   7128,    348,     19,    284,    220,
             19,  20761,     31,     26,     31,  12889,   7128,    348,     20,
            284,    220,    

In [31]:
output = hf_llm.generate(inputs=input_tokens.cuda(), do_sample=False, max_new_tokens=512)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(output_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



System: You are an assistant. 
User: Hello! I want to test your mathematic ability. Your task is to finish the mathematical task. The problem for you is:
Condition 1 : @<<<v0 = 1>>>@;@<<<assign v1 = 1>>>@;
Condition 2 : @<<<assign v2 = 5>>>@;@<<<assign v3 = 2>>>@;
Condition 3 : @<<<assign v4 = 4>>>@;@<<<assign v5 = 1>>>@;
**Question:**
Can you tell me how many variables are equal to 1 in those assignments in '<<<>>>'? The values of those variables can be used before definition as they are constant. 
Please solve the problem in a step-by-step way.
Assistant: 
I'd be happy to help you with that. Let's break down the problem step by step.

From the given conditions, we have:

Condition 1: v0 = 1 and v1 = 1
Condition 2: v2 = 5 and v3 = 2
Condition 3: v4 = 4 and v5 = 1

Now, let's identify the variables that are equal to 1:

1. v0 = 1 (from Condition 1)
2. v1 = 1 (from Condition 1)
3. v5 = 1 (from Condition 3)

There are three variables that are equal to 1.
